In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
project_path = Path.cwd().parent
sys.path.append(project_path/'scripts')
from scripts.data_utils import DataLoader
from scripts.model_utils import XGBoostTrainer
from scripts.model_utils import (
    create_hierarchical_pipelines, train_hierarchical_pipelines, 
    save_pipelines, load_hierarchical_pipeline_and_predict, evaluate)

### 0 - Load Data

In [4]:
data_path = Path.home() / 'data/craig_pfc_2023/step_2_cleaned'
assert data_path.exists()

In [8]:
data_path

PosixPath('/Users/erdemkarakoylu/data/craig_pfc_2023/step_2_cleaned')

In [5]:
loader = DataLoader(data_path=data_path, rrs_file = 'df_rrs.pqt', phy_file='df_phy.pqt')

2024-09-20 14:25:39.241 | DEBUG    | scripts.data_utils:__init__:16 - Data directory set to /Users/erdemkarakoylu/data/craig_pfc_2023/step_2_cleaned
2024-09-20 14:25:39.241 | DEBUG    | scripts.data_utils:__init__:17 - Rrs file used: df_rrs.pqt
2024-09-20 14:25:39.241 | DEBUG    | scripts.data_utils:__init__:18 - Phytoplankton file use df_phy.pqt


In [6]:
dX, dY =  loader.load_data()

In [7]:
dX.head().T

,9,10,11,12,13
250,0.001732,0.001895,0.001962,0.001969,0.001942
251,0.001731,0.001894,0.001962,0.001968,0.001941
252,0.001729,0.001893,0.001961,0.001967,0.001940
253,0.001727,0.001891,0.001959,0.001966,0.001939
254,0.001725,0.001889,0.001957,0.001964,0.001936
...,...,...,...,...,...
746,0.000028,0.000009,0.000009,0.000009,0.000009
747,0.000029,0.000010,0.000027,0.000009,0.000027
748,0.000029,0.000029,0.000009,0.000009,0.000009
749,0.000020,0.000010,0.000027,0.000027,0.000027


In [6]:
dX_train, dX_test, dY_train, dY_test = train_test_split(
    dX, dY
)

In [7]:
dY_train.columns.tolist()

['dia', 'chl', 'cya', 'coc', 'din', 'pha', 'tot_cphyl']

In [7]:
pipelines = create_hierarchical_pipelines(dY_train)

In [8]:
params = {
    "objective": "reg:squarederror", # Basic parameter
    "n_estimators": 30,             # Small number for faster testing
    "seed": 42,                      # Reproducibility
    "learning_rate": 0.1             # For reproducibility
    }
train_hierarchical_pipelines(pipelines, dX_train, dY_train, params)

: 

In [15]:
dY_train.mean(axis=1).shape

(946205,)

In [8]:
output_vars = dY_train.columns.to_list()

In [9]:
output_vars

['dia', 'chl', 'cya', 'coc', 'din', 'pha', 'tot_cphyl']

In [10]:
params = {
        'objective': 'reg:squarederror', 
        'n_estimators': 100,
        'learning_rate': 0.1,
        # Add other parameters as needed
    }
pipelines = create_hierarchical_pipelines(dY_train) 
train_hierarchical_pipelines(pipelines, dX_train, dY_train, params)
save_pipelines(pipelines)

    # Example prediction and evaluation for one output variable with uncertainty
output_var = 'tot_cphyl'
mean_pred, std_pred = load_hierarchical_pipeline_and_predict(output_var, dX_test)


ValueError: y must have at least two dimensions for multi-output regression but has only one.